# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770361007556                   -0.52    8.0    156ms
  2   -2.771687187476       -2.88       -1.32    1.0    116ms
  3   -2.771714304243       -4.57       -2.49    1.0   96.5ms
  4   -2.771714651370       -6.46       -3.27    1.0    102ms
  5   -2.771714712676       -7.21       -3.82    2.0    115ms
  6   -2.771714715165       -8.60       -4.96    1.0    104ms
  7   -2.771714715235      -10.15       -4.97    2.0    122ms
  8   -2.771714715249      -10.84       -6.09    1.0    112ms
  9   -2.771714715250      -12.54       -6.95    2.0    128ms
 10   -2.771714715250   +  -13.95       -7.36    1.0    113ms
 11   -2.771714715250      -14.40       -8.28    1.0    115ms


-0.00013457372503162835

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770391164897                   -0.53    9.0    169ms
  2   -2.771769490581       -2.86       -1.30    1.0    121ms
  3   -2.771801579935       -4.49       -2.62    1.0    114ms
  4   -2.771802047356       -6.33       -4.00    1.0    103ms
  5   -2.771802074121       -7.57       -4.28    2.0    119ms
  6   -2.771802074445       -9.49       -5.05    1.0    108ms
  7   -2.771802074474      -10.54       -5.42    2.0    126ms
  8   -2.771802074476      -11.71       -6.54    1.0    111ms
  9   -2.771802074476      -13.30       -6.79    2.0    126ms
 10   -2.771802074476      -14.24       -7.03    1.0    114ms
 11   -2.771802074476      -13.85       -7.83    1.0    116ms
 12   -2.771802074476   +  -14.51       -8.21    2.0    930ms


0.017612221450990603

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457372503162835
Displaced dipole:  0.017612221450990603
Polarizability :   1.774679517602223


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  70.4ms  Non-interacting
   1        0       1       -0.60     10.0   1.12s  
   2        0       2       -2.42      7.0   120ms  
   3        0       3       -3.54      5.0   158ms  
   4        0       4       -5.33      4.0  96.7ms  
   5        0       5       -7.28      1.0   109ms  
   6        0       6      -10.29      1.0  83.1ms  
   7        1       1       -7.37     13.0   224ms  Restart
   8        1       2       -8.99      1.0  82.4ms  
                                      13.0   128ms  Final orbitals


(δψ = Matrix{ComplexF64}[[-0.0032732265697002234 + 0.0003858901432221787im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.033643459719478644 + 0.30756188396811895im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; 0.004643555614686006 + 0.04080176750757118im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.009784251907332316 - 0.0803446347714644im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206952072427465e-7 -2.5020162667286796e-7 … -4.880641396579033e-8 -2.5020150674899313e-7; -6.293618312043701e-7 -4.793038208669898e-7 … -1.2077846523638187e-7 -4.793037226060664e-7; … ; 1.3626269326258324e-7 1.1356240719932574e-7 … 4.711997218064555e-8 1.1356237679041156e-7; 1.33054801181863e-7 1.3632641566652855e-7 … 5.200689514567188e-8 1.3632648147525917e-7;;; -2.502015972775746e-7 -1.74049190159058e-7 … -3.556494082999387e-8 -1.7404909658733882e-7; -4.79303810040029e-7 -3.6797338883043755e-7 … -1.1202899740540678e-7 -3.679733040006143e-7; … ; 1.1356239591711825e-7 1.1211110688507303e-7 … 1.0980694685448096e-7 1.121

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125335405985
Interacting polarizability:     1.7736548631471574


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 7.91e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.06e-11
[ Info: GMRES linsolve in iteration 2; step 2: normres = 4.45e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 2.85e-13
└ * number of operations = 12
Non-interacting polarizability: 1.9257125335405125
Interacting polarizability:     1.7736548604385602


We obtain the identical result to above.